In [ ]:
!pip install -q pytorch-tabnet scikit-learn pandas

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

np.random.seed(42)
num_samples = 500
df = pd.DataFrame({
    "src_port": np.random.randint(1000, 65535, size=num_samples),
    "dst_port": np.random.randint(20, 1024, size=num_samples),
    "proto": np.random.choice([6, 17], size=num_samples),
    "pkt_len": np.random.randint(64, 1500, size=num_samples),
    "ttl": np.random.randint(1, 255, size=num_samples),
    "flag": np.random.choice(["SYN", "ACK", "FIN", "PSH"], size=num_samples),
    "label": np.random.choice(["normal", "intrusion"], size=num_samples)
})

label_enc = LabelEncoder()
df["flag_enc"] = label_enc.fit_transform(df["flag"])
df["label_enc"] = label_enc.fit_transform(df["label"])
df.drop(columns=["flag", "label"], inplace=True)

X = df.drop("label_enc", axis=1).values
y = df["label_enc"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(" Random Forest Accuracy:", accuracy_score(y_test, y_pred))

clf = TabNetClassifier()
clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=10,
    patience=5,
    batch_size=32,
    virtual_batch_size=16,
    num_workers=0
)

tabnet_preds = clf.predict(X_test)

if hasattr(y_test, 'values'):
    y_test = y_test.values.ravel()

print(" Predictions ready!")
print(" Shape of predictions:", tabnet_preds.shape)
print(" Shape of y_test:", y_test.shape)

print(" TabNet Accuracy:", accuracy_score(y_test, tabnet_preds))
print(classification_report(y_test, tabnet_preds, target_names=["normal", "intrusion"]))
